In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model, StatsTracker
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES, get_bt_index

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import encode_target, FGW
from mol_opt.train_mol_opt import ft

from rdkit.Chem import MolFromSmiles

from molgen.metrics.Penalty import Penalty

import torch
from torch import nn
import numpy as np
import time

/Users/octavdragoi/gitrepos/tum-thesis


In [2]:
sys.argv = ["", "-cuda"]
args = get_args()
model = "pointwise10-onebatch"
args.output_dir = f"mol_opt/output_{model}/"

model_iter = 1000 

model_name = "model_{}_{}".format(model, model_iter)
model_decode_name = "model_{}_decode_{}".format(model, model_iter)

molopt, config = load_model(args.output_dir + model_name, MolOpt, args.device)
print (molopt, config)

molopt_decoder, config_decoder = load_model(args.output_dir + model_decode_name, MolOptDecoder, args.device)
print (molopt_decoder, config_decoder)

loss = FGW(alpha = 0.5)

n_data = 36
data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", n_data, True)
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

FileNotFoundError: [Errno 2] No such file or directory: 'mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_1000'

In [4]:
args.conn_penalty_function = 'capped_logdet2'
pen = PenaltyNew(args, prev_epoch = 0)

In [5]:
yhat_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(yhat_embedding, X, Y)
yhat_labels = molopt_decoder.discretize(*yhat_logits)

pred_pack = (yhat_labels, yhat_logits, Y.scope), Y

In [54]:
for idx in range(50):
    stats_tracker = StatsTracker()
    con_loss, val_loss, eul_loss = pen(*pred_pack, idx)
    stats_tracker.add_stat('conn_penalty', con_loss.item(), n_data)
    stats_tracker.add_stat('val_penalty', val_loss.item(), n_data)
    stats_tracker.add_stat('euler_penalty', eul_loss.item(), n_data)
    
    stats_tracker.print_stats("epoch={}".format(idx))
    pen.log()

epoch=0
 conn_penalty:0.0000004
 val_penalty:1.7847447
 euler_penalty:25.6435683
Penalty params: tau=1.00000 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=1
epoch=1
 conn_penalty:0.0000004
 val_penalty:1.5047135
 euler_penalty:25.6922082
Penalty params: tau=1.00000 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=1
epoch=2
 conn_penalty:0.0000004
 val_penalty:1.4603640
 euler_penalty:25.8834195
Penalty params: tau=0.99990 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=2
epoch=3
 conn_penalty:0.0000004
 val_penalty:1.5447070
 euler_penalty:25.5283271
Penalty params: tau=0.99980 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=3
epoch=4
 conn_penalty:0.0000004
 val_penalty:0.9759809
 euler_penalty:26.2329356
Penalty params: tau=0.99970 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=4
epoch=5
 conn_penalty:0.0000004
 val_penalty:1.8451453
 euler_penalty:25.8235575
Penalty params: tau=0.99960 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=5
epoch=6
 conn_penalty:0.0000

In [55]:
PenaltyNew(args, prev_epoch = 0).log()

Penalty params: tau=1.00000 conn_l=0.02500 val_l=0.07000 euler_l=0.30000 epoch=1


In [86]:
adjM = pen(*pred_pack, idx)

In [112]:
# construct adjacency matrix from this thing
adjM

tensor([[1.1921e-06, 1.3630e-02, 1.0782e-04, 4.4374e-03, 1.8051e-01, 8.4739e-02,
         0.0000e+00, 8.5733e-02, 9.3007e-04, 1.4305e-06, 0.0000e+00, 2.6822e-06,
         0.0000e+00, 5.4988e-01, 1.2422e-02, 6.8890e-01, 3.5245e-01, 7.2306e-04,
         2.2233e-05, 5.0822e-01, 2.1683e-01, 1.0255e-02],
        [1.3630e-02, 3.9140e-03, 5.3970e-01, 2.2346e-03, 3.8520e-02, 8.8930e-04,
         2.3283e-01, 4.9831e-01, 2.1935e-05, 3.9616e-01, 5.9605e-08, 7.7458e-01,
         2.7418e-06, 7.3087e-02, 5.0057e-01, 2.1463e-01, 2.4846e-02, 6.6979e-02,
         1.3762e-01, 1.4181e-01, 3.7023e-02, 7.1169e-02],
        [1.0782e-04, 5.3970e-01, 7.7593e-02, 3.8449e-01, 9.0742e-02, 1.0008e-01,
         1.4413e-01, 2.2877e-01, 5.5884e-03, 2.4866e-02, 0.0000e+00, 3.4104e-02,
         0.0000e+00, 1.4942e-02, 6.2261e-02, 2.6948e-02, 6.4849e-02, 6.2976e-02,
         4.5100e-01, 3.7974e-01, 8.5898e-01, 2.9276e-01],
        [4.4374e-03, 2.2346e-03, 3.8449e-01, 1.2787e-03, 3.2146e-01, 3.9279e-05,
         1.8100e

In [41]:
import numpy as np
np.random.randint(100)

75

In [64]:
bonds = Y.get_graph_outputs()[0]["BOND_TYPES"]

In [15]:
adjM = 1 - bonds[:,:,-1]

In [90]:
pen.conn_penalty(adjM)

tensor(0., device='cuda:0', grad_fn=<AddBackward0>)

In [92]:
N = adjM.shape[0]
device = adjM.device

# get Laplacian
L = torch.diag(torch.matmul(adjM, torch.ones(N, device=device))) - adjM
L_mod = L + torch.ones_like(L, device=device) / N

# calculate log dets
# comment this line to use the rescaling procedure
# return (-torch.logdet(L_mod + self.conn_eps * torch.eye(N, device=device)))

# calculate rescaled eigenvalues
eigvals = torch.symeig(L_mod, eigenvectors = True)[0]
torch.sum(torch.exp(-eigvals))
        

tensor(2.6787, device='cuda:0', grad_fn=<SumBackward0>)

In [108]:
cap_value = 1e-02
conn_eps = 1e-09

eigenvals = torch.symeig(L_mod + conn_eps * torch.eye(N, device=device))[0]
-torch.sum(torch.log(eigenvals.clamp(max = cap_value))) + len(eigvals) * np.log(cap_value)

tensor(0., device='cuda:0', grad_fn=<AddBackward0>)

In [94]:
eigenvals

tensor([1.0000e-09, 7.2240e-01, 1.0000e+00, 1.3127e+00, 1.5558e+00, 2.1292e+00,
        2.2047e+00, 2.3685e+00, 2.4458e+00, 2.6390e+00, 2.7848e+00, 3.0165e+00,
        3.1404e+00, 3.4076e+00, 3.6440e+00, 3.8691e+00, 4.0975e+00, 4.2596e+00,
        4.3029e+00, 4.5827e+00, 4.8254e+00, 5.2925e+00], device='cuda:0',
       grad_fn=<CopySlices>)

In [95]:
eps = 1e-9
beta = 1e-3
- torch.sum(torch.log(eigvals.clamp(min = eps, max = beta))) + len(eigvals) * np.log(beta)

tensor(0., device='cuda:0', grad_fn=<AddBackward0>)

In [102]:
eigvals

tensor([0.5753, 0.7224, 1.0000, 1.3127, 1.5558, 2.1292, 2.2047, 2.3685, 2.4458,
        2.6390, 2.7848, 3.0165, 3.1404, 3.4076, 3.6440, 3.8691, 4.0975, 4.2596,
        4.3029, 4.5827, 4.8254, 5.2925], device='cuda:0',
       grad_fn=<SymeigBackward>)

In [103]:
eigenvals

tensor([0.5753, 0.7224, 1.0000, 1.3127, 1.5558, 2.1292, 2.2047, 2.3685, 2.4458,
        2.6390, 2.7848, 3.0165, 3.1404, 3.4076, 3.6440, 3.8691, 4.0975, 4.2596,
        4.3029, 4.5827, 4.8254, 5.2925], device='cuda:0',
       grad_fn=<SymeigBackward>)

In [5]:
torch.symeig(L_mod, eigenvectors = True)

NameError: name 'L_mod' is not defined

In [91]:
args = get_args()

args.scale_lambdas = True
args.conn_lambda_start = 0.1 
args.conn_lambda_end = 25
args.conn_lambda_epochs_start = 20
args.conn_lambda_epochs_end = 50
args.valency_lambda_start = 1
args.valency_lambda_end = 100
args.valency_lambda_epochs_start = 50
args.valency_lambda_epochs_end = 60
args.euler_characteristic_penalty = False
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 40

pen = Penalty(args)

pen.log()
for i in range(1, 100):
    pen.compute_lambdas(i)
    pen.log()

Penalty params: tau=1.00000 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=0 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.88862 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=1 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.78965 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=2 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.70170 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=3 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.62355 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=4 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.55410 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=5 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.49239 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=6 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.43755 conn_l=0.10000 val_l=1.00000 euler_l=1.00000 epoch=7 mode=[0 0 0] conn=False euler=

In [87]:
args = get_args()

args.scale_lambdas = True
# args.connectivity = True 
# args.valency = True 
# args.euler_characteristic_penalty = False

args.conn_lambda_start = 1 
args.conn_lambda_end = 25
args.conn_lambda_epochs_start = 4000
args.conn_lambda_epochs_end = 8000
args.valency_lambda_start = 1
args.valency_lambda_end = 100
args.valency_lambda_epochs_start = 7000
args.valency_lambda_epochs_end = 10000
args.euler_characteristic_penalty = False
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 10000

pen = Penalty(args)

pen.log()
for i in range(1, 100):
    pen.compute_lambdas(i)
    pen.log()

Penalty params: tau=1.00000 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=0 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99954 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=1 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99908 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=2 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99862 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=3 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99816 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=4 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99770 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=5 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99724 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=6 mode=[0 0 0] conn=False euler=False val=False
Penalty params: tau=0.99678 conn_l=1.00000 val_l=1.00000 euler_l=1.00000 epoch=7 mode=[0 0 0] conn=False euler=

In [62]:
(pen.tau_end / pen.tau_start) ** (1. / (pen.tau_epochs_end - pen.tau_epochs_start))

0.7847599703514613